# GLOBAL ALIGNMENT

In [ ]:
from pandas.core.api import DataFrame

import numpy as np    
import pandas as pd                                        
import string
def multiples(value, length):
    return [value * i for i in range(1, length + 1)] 
  
def subs( word1: str, word2: str,gap = -2) -> int:
    m,n = len(list(word1)),len(list(word2))
    #m1 = set(list(word1)+list(word2))
    n_indel = multiples(-2,n) #vertical
    m_indel = multiples(-2,m)
    dp  = [ [""]+[""]+list(word1)]+[[""]+[0]+m_indel] +[[r]+[n_indel[i]]+[0]*m for i,r in enumerate(list(word2))] #m is actually becoming columns as being a entire row, and n is actually becoming row , as being entire column
    m1 = set(list(word1)+list(word2))
   
    dp1  = [ [""]+[""]+list(m1)]+[[""]+[0]+m_indel] +[[r]+[n_indel[i]]+[0]*n for i,r in enumerate(list(m1))] #m is actually becoming columns as being a entire row, and n is actually becoming row , as being entire column
   
    dp = pd.DataFrame(dp)
    dp1 = pd.DataFrame(dp1)
    dp[0][0]=0
    dir=[]
   
    t = word1  # I have assumes word1 as the one in rows , and word2 as one in columns
    word1 = word2
    word2 = t
    for i in range(2,m+2): # for every cell
        temp = []
        for j in range(2,n+2):
          
          if(list(word1)[i-2] == list(word2)[j-2]):           #ith row of m length word, jth column of n length word , the index of the word is actually twice far behind
                
              
              #dp[i][j] = dp[i-1][j-1]+1
              dp.iloc[i,j:j+1] = dp.iloc[i-1,j-1:j] +1
              temp.append("D")
          else:
              ind = np.array([dp.iloc[i,j-1:j]+gap,dp.iloc[i-1,j:j+1]+gap,dp.iloc[i-1,j-1:j]-1])
              dp.iloc[i,j:j+1]  =  max(ind) # +1 for edit distance
              #ind = np.array([dp[i][j-1]-2,dp[i-1][j]-2,dp[i-1][j-1]-1])
              #dp[i][j] =  max([dp[i][j-1]-2,dp[i-1][j]-2,dp[i-1][j-1]-1]) # +1 for edit distance
              
              max_t = np.where(ind == np.max(ind))[0]
             
               
              if(len(max_t) > 1) :
                tup = ()
                for ind1 in max_t:
                  
                  if ind1 == 0:
                    tup = tup + tuple("L")
                  elif ind1 == 1:
                    tup = tup + tuple("T")
                    
                  else:
                    tup = tup + tuple("D")
                    
                temp.append(sorted(tup))
              else:
                if max_t == 0:
                    temp.append("L")
                elif max_t == 1:
                    temp.append("T")
                else:
                    temp.append("D")
        dir.append(temp)
    dp.drop(columns=[1], axis=1,inplace = True)
    dp.drop(index=[1], axis=0,inplace = True)
    #dp = dp.reset_index(drop=True)
    dp1 = pd.DataFrame(dp.to_numpy())
    
    d={}
    save=[]
   
    from collections import defaultdict
    d={}
    for i,j in dp1.iterrows():
      k = list(j)
      if (i == 0):
          
          
          del k[0]
          for i,kk in enumerate(k):
              k[i] = kk+" "+str(i)
          save = k
         
        
      else: 
        res= {}
        #print(k[0],list(zip(save,k[1:])))
        res.update(list(zip(save,k[1:])))
        d.update([(k[0]+" "+str(i),res)])
    
    alphabet_string = string.ascii_lowercase
    alphabet_list = list(alphabet_string)
   
    return (d,pd.DataFrame(dir))
word1 = "ctac"
word2 = "gata"
gap =-2
substitution,dir = subs(word1,word2)
print(substitution)

{'g 1': {'c 0': -1, 't 1': -3, 'a 2': -5, 'c 3': -7}, 'a 2': {'c 0': -3, 't 1': -2, 'a 2': -2, 'c 3': -4}, 't 3': {'c 0': -5, 't 1': -2, 'a 2': -3, 'c 3': -3}, 'a 4': {'c 0': -7, 't 1': -4, 'a 2': -1, 'c 3': -3}}


In [ ]:
import itertools
class Solution:
 def global_alignment(self, sequence_A: str, sequence_B:str, substitution: dict, gap: int ) -> [tuple]:
 
    save = []
    word1 = sequence_A
    word2 = sequence_B
    res1 = pd.DataFrame(substitution)    
    
    res1 = res1.T     
   
    res1.columns = [r.split(" ")[0] for r in res1.columns]
    res1.index = [r.split(" ")[0] for r in res1.index]
    
    dp = res1
    #dp = dp.to_numpy()
    substitution = subs(word1,word2,gap)[0]
    dir = subs(word1,word2,gap)[1]
    size = np.shape(dp)
    size1 = np.shape(dir)
    
    curr = dir.iloc[size1[0]-1,size1[1]-1:size1[1]]
    curr = curr.iloc[0]
    
    ind = [size1[0]-1,size1[1]-1]
    res =()
    res1=[]
    t1 = list(word2)
    t2 = list(word1)
    
    path1 =""
    path2 =""
    
    next = curr
    path = False
   
    while( path == False):
            next =  curr
            if(str(type((next))) == "<class 'list'>"):
            
              
                if next == ["D","L"]: #horizontal word1 path1 has two paths now 
              
                  t = [path1 + "_", path1+t1[ind[0]]]
                  path1 =t
                  path2 = path2 + t2[ind[1]] #corresponding column word2 #next going left, reduce column
                  if(ind[0] == 0 and ind[1] == 0):
                    path = True
                  if(ind[1] - 1 >= 0):
                    ind[1] = ind[1]-1 
                    curr = dir.iloc[ind[0],ind[1]:ind[1]+1]
                    curr = curr.iloc[0]
              
                elif next == ["D","T"]: #vertical word2 jth column path2 has two paths now
                 
                  t = [path2 + "_",path2+t2[ind[1]]] # _ for word2 column
                  path2 =t
                  path1 = path1 + t1[ind[0]] # corresponding row t1 word2 #next going top, reduce row
                  if(ind[0] == 0 and ind[1] == 0):
                   path = True
                  if(ind[0] - 1 >= 0):
                    
                    ind[0] = ind[0]-1
                    curr = dir.iloc[ind[0],ind[1]:ind[1]+1]
                    curr = curr.iloc[0]
                    
            else:
              if next == "D":
                if(str(type(path1)) =="<class 'list'>"):
                   
                    path1[0] = path1[0]+ t1[ind[0]]
                    path1[1] = path1[1]+ t1[ind[0]]
                    path2 = path2+t2[ind[1]]
                elif(str(type(path2)) =="<class 'list'>"):
                    path2[0] = path2[0]+ t2[ind[1]]
                    path2[1] = path2[1]+ t2[ind[1]]
                    path1 = path1+t1[ind[0]]
                else:
                    path1 = path1+t1[ind[0]]
                    path2 = path2+t2[ind[1]]
               
                if(ind[0] == 0 and ind[1] == 0):
                   path = True
                
                if(ind[1] - 1 >= 0 and ind[0]-1 >= 0):
                  ind[1] = ind[1]-1 
                  ind[0] = ind[0]-1
                  
                  curr = dir.iloc[ind[0],ind[1]:ind[1]+1]
                  curr = curr.iloc[0]
                  
              elif next =="L":
                  path1 = path1 +"_"
                  path2 = path2 +t2[ind[1]] 
                  if(ind[0]-1 == 0 and ind[1] == 0):
                      path = True 
                  if(ind[1] - 1 >= 0):
                  
                      
                      ind[1] = ind[1] -1 
                      curr = dir.iloc[ind[0],ind[1]:ind[1]+1]
                      curr = curr.iloc[0]
               
              else:
                path2 = path2 + "_"
                path1 = path1 + t1[ind[0]]
                if(ind[0] == 0 and ind[1] == 0):
                  path = True
                if(ind[0]-1  >= 0):

                  
                 ind[0]=ind[0] - 1
                 curr = dir.iloc[ind[0],ind[1]:ind[1]+1]
                 curr = curr.iloc[0]
    
    if(str(type((path1))) == "<class 'list'>" and str(type((path2))) == "<class 'list'>"):
      path1 = [''.join(reversed(p)) for p in path1]
      path2 = [''.join(reversed(p)) for p in path2]
      #print(list(itertools.product(path1,path2)))
      res = list(itertools.product(path1,path2))
      #return list(itertools.product(path1,path2))
    if(str(type((path1))) == "<class 'list'>"):
        path1 = [''.join(reversed(p)) for p in path1]
        #print(list(itertools.product(path1,[''.join(reversed(path2))])))
       # return list(itertools.product(path1,[''.join(reversed(path2))]))
        res = list(itertools.product(path1,[''.join(reversed(path2))]))
    elif(str(type((path2))) == "<class 'list'>"):
        path2 = [''.join(reversed(p)) for p in path2]
        #print(list(itertools.product(path2,[''.join(reversed(path1))])))
        #return list(itertools.product(path1,[''.join(reversed(path2))]))
        res = list(itertools.product(path2,[''.join(reversed(path1))]))
    else:
      print(''.join(reversed(path1)),''.join(reversed(path2)))
      return [(''.join(reversed(path1)),''.join(reversed(path2)))]
    return res

    def _main_():
      gap = -2 
      global_alignment(self,word1,word2,substitution,gap)

c= Solution
gap = -2
c.global_alignment(c,word1,word2,subs(word1,word2)[0],gap)



[('c_tac', 'gata_'), ('cctac', 'gata_')]

 

# LOCAL ALIGNMENT



In [ ]:
 m,n = len(list(word1)),len(list(word2))
 m_indel = [0]*m
 n_indel = [0]*n #vertical
 dp  = [ [0]+[0]+list(word1)]+[[0]+[0]+m_indel] +[[r]+[n_indel[i]]+[0]*m for i,r in enumerate(list(word2))] #m is actually becoming columns as being a entire row, and n is actually becoming row , as being entire column
 print(pd.DataFrame(dp))

   0  1  2  3  4  5
0  0  0  c  t  a  c
1  0  0  0  0  0  0
2  g  0  0  0  0  0
3  a  0  0  0  0  0
4  t  0  0  0  0  0
5  a  0  0  0  0  0


In [ ]:
from pandas.core.api import DataFrame

import numpy as np    
import pandas as pd                                        
import string
def multiples(value, length):
    return [value * i for i in range(1, length + 1)] 



  
def subs1( word1: str, word2: str,gap=-2) -> int:
  if(len(word1) > len(word2)):
      m,n = len(list(word2)),len(list(word1)) #the shorter word becomes the row which is 'n', and  i in for loop
  else: #longer word becomes the column which is 'm'
      m,n = len(list(word1)),len(list(word2))
 
  m_indel = [gap]*m
  n_indel = [gap]*n #vertical
  dp  = [ [0]+[0]+list(word1)]+[[0]+[0]+m_indel] +[[r]+[n_indel[i]]+m_indel for i,r in enumerate(list(word2))] #m is actually becoming columns as being a entire row, and n is actually becoming row , as being entire column
  m1 = set(list(word1)+list(word2))
  
  
  dp = pd.DataFrame(dp)
  dp[0][0]=0
  
  t = word1  # I have assumes word1 as the one in rows , and word2 as one in columns
  word1 = word2
  word2 = t
  for i in range(2,m+2): # for every cell
    temp = []
    for j in range(2,n+2):
      
                 #ith row of m length word, jth column of n length word , the index of the word is actually twice far behind
         #print(list(word1)[i-2], list(word2)[j-2])
         if(list(word1)[i-2] == list(word2)[j-2]):           #   ith row of m length word, jth column of n length word , the index of the word is actually twice far behind
                
              
              #dp[i][j] = dp[i-1][j-1]+1
              dp.iloc[i,j:j+1] = dp.iloc[i-1,j-1:j] +2
              temp.append("D")
         else:
              ind = np.array([dp.iloc[i,j-1:j]+gap,dp.iloc[i-1,j:j+1]+gap,dp.iloc[i-1,j-1:j]-1])# g is used here
              dp.iloc[i,j:j+1]  = max([max(ind),0])# +1 for edit distance
        
            
            
             
  dp.drop(columns=[1], axis=1,inplace = True)
  dp.drop(index=[1], axis=0,inplace = True)
     
  dp1 = pd.DataFrame(dp.to_numpy())
  

  save=[]

  from collections import defaultdict
  d={}
  for i,j in dp1.iterrows():
    k = list(j)
    
    if (i == 0):
        
        
        
        del k[0]
        for i,kk in enumerate(k):
              k[i] = kk+" "+str(i)
        save = k
       
      
    else: 
      res= {}
      
      res.update(list(zip(save,k[1:])))
      d.update([(k[0]+" "+str(i),res)])
      
   
  
  return d           

alphabet_string = string.ascii_lowercase
alphabet_list = list(alphabet_string)        
    

word1 = "ctac"
word2 = "gata"
#word1= "thequickbrownfoxjumpsoverthelazydog"
#word2= "krithikarajendran"
word1= "thequickbrownfoxjumpsoverthelazydog"
word2= "krithikarajendran"
substitution1= subs1(word1,word2)

#print(
res1 = pd.DataFrame(substitution1)    

res1 = res1.T     
res1.columns = [r.split(" ")[0] for r in res1.columns]
res1.index = [r.split(" ")[0] for r in res1.index]
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)
pd.set_option('display.colheader_justify', 'center')
pd.set_option('display.precision', 3)

display(res1)




/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,t,h,e,q,u,i,c,k,b,r,o,w,n,f,o,x,j,u,m,p,s,o,v,e,r,t,h,e,l,a,z,y,d,o,g
k,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
r,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0
i,0,0,0,0,0,2,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
t,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0
h,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4,2,0,0,0,0,0,0,0
i,0,0,1,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,3,1,0,0,0,0,0,0
k,0,0,0,0,0,0,1,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,2,0,0,0,0,0,0
a,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4,2,0,0,0,0
r,0,0,0,0,0,0,0,0,0,3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,2,3,1,0,0,0
a,0,0,0,0,0,0,0,0,0,1,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,2,1,2,0,0,0


In [ ]:

  m_indel = [0]*m
  n_indel = [0]*n #vertical
  dp  = [ [0]+[0]+list(word1)]+[[0]+[0]+m_indel] +[[r]+[n_indel[i]]+[0]*m for i,r in enumerate(list(word2))] 

In [ ]:
from tabulate import tabulate
import itertools
class Solution:
 def local_alignment(self, sequence_A: str, sequence_B:str, substitution: dict, gap: int ) -> [tuple]:
 
    save = []
    word1 = sequence_A
    word2 = sequence_B
    
    res1 = pd.DataFrame(subs1(word1,word2,gap))    
    
    res1 = res1.T     
    t = reversed(list(word1))
    res1.columns = [r.split(" ")[0] for r in res1.columns]
    res1.index = [r.split(" ")[0] for r in res1.index]
    display(res1.columns)
    display(res1)
    print(tabulate(pd.DataFrame(res1),tablefmt='psql'))
    m=[]
    path1=""
    path2=""
    flip = 0
    c =0
    prev = 0
    matches = []
    path = ""
    semi =[]
    count = res1.shape[0]-1
    for i,j in reversed(list(res1.iterrows())):
      
      max1 = list(j).index(max(list(j)))
      mx = max(list(j))
     
      if c > 0:
         
         if(pr - mx == 2 and prev - max1 == 1):
            if(flip == 1 ): #to assure a new path is building(already there was a match) because it is 4-2, 2-0 not simply by chance but because of character match before
              if len(list(path)) > 1: #to check list(path) > 1 ,this can first time in this place, or it has been here before and now comes after a gap
                
                matches.append((path,path))  #appending the old path
              #semi-matches
              semi.pop() 
              path = ""+list(res1.columns)[prev] #creating a new path as old path is stored
              flip = 0
            path = path +list(res1.columns)[max1]
         else:
            if(list(res1.index)[count] == list(res1.columns)[max1]): #new pattern starting
              
              semi.append(list(res1.columns)[max1])
              flip = 1
            else:
              flip =0
            if(pr == 2):
                res1.iloc[count+1,prev:prev+1] = 1  #semi-matches
                
            
            
      else:
           if(list(res1.index)[count] == list(res1.columns)[max1]):
                semi.append(list(res1.columns)[max1])
            
                flip = 1
           else: 
               flip == 0
          

     
   
      prev =max1
      pr = mx    
      c = c + 1
      count= count-1
    if(len(list(path)) > 1):
       matches.append((path,path))
    if(flip == 1):
               
               #print( res1.iloc[count+1,prev:prev+1])
               res1.iloc[count+1,prev:prev+1] = 1  

    print(matches)
    print(semi)

    return matches,res1
    def _main_():
      gap = -2 
      local_alignment(self,word1,word2,substitution,gap)

c= Solution
word1= "thequickbrownfoxjumpsoverthelazydog"
word2= "krithikarajendran"
gap = -2
result,res_d = c.local_alignment(c,word1,word2,subs1(word1,word2),gap)

print(result)
display(res_d)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Index(['t', 'h', 'e', 'q', 'u', 'i', 'c', 'k', 'b', 'r', 'o', 'w', 'n', 'f', 'o', 'x', 'j', 'u', 'm', 'p', 's', 'o', 'v', 'e', 'r', 't', 'h', 'e', 'l', 'a', 'z', 'y', 'd', 'o', 'g'], dtype='object')

,t,h,e,q,u,i,c,k,b,r,o,w,n,f,o,x,j,u,m,p,s,o,v,e,r,t,h,e,l,a,z,y,d,o,g
k,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
r,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0
i,0,0,0,0,0,2,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
t,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0
h,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4,2,0,0,0,0,0,0,0
i,0,0,1,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,3,1,0,0,0,0,0,0
k,0,0,0,0,0,0,1,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,2,0,0,0,0,0,0
a,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4,2,0,0,0,0
r,0,0,0,0,0,0,0,0,0,3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,2,3,1,0,0,0
a,0,0,0,0,0,0,0,0,0,1,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,2,1,2,0,0,0


+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+
| k | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 |
| r | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 2 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 2 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 |
| i | 0 | 0 | 0 | 0 | 0 | 2 | 0 | 0 | 0 | 0 | 1 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 1 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 |
| t | 0 | 0 | 0 | 0 | 0 | 0 | 1 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 2 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 |
| h | 0 | 2 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 4 | 2 | 0 | 0 | 0 | 0 | 0 | 0 | 0 |
| i | 0 | 0 | 1 | 0 | 0 | 2 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 2 | 3 | 1 | 0 

,t,h,e,q,u,i,c,k,b,r,o,w,n,f,o,x,j,u,m,p,s,o,v,e,r,t,h,e,l,a,z,y,d,o,g
k,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
r,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0
i,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
t,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
h,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4,2,0,0,0,0,0,0,0
i,0,0,1,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,3,1,0,0,0,0,0,0
k,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,2,0,0,0,0,0,0
a,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4,2,0,0,0,0
r,0,0,0,0,0,0,0,0,0,3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,2,3,1,0,0,0
a,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,2,1,2,0,0,0


+---+---+----+----+----+----+----+
| 0 | 0 |    | c  | t  | a  | c  |
| 1 |   | 0  | -2 | -4 | -6 | -8 |
| 2 | g | -2 | -1 | -3 | -5 | -7 |
| 3 | a | -4 | -3 | -2 | -2 | -4 |
| 4 | t | -6 | -5 | -2 | -3 | -3 |
| 5 | a | -8 | -7 | -4 | -1 | -3 |
+---+---+----+----+----+----+----+
